In [1]:
import data.breathe_data as bd
import pandas as pd
import data.helpers as dh
import pandas as pd

## Analyse Age

In [9]:
patient_df = pd.read_csv(
    dh.get_path_to_main() + "DataFiles/BR/processedRedCapData/brPatient_20240509.csv",
    # usecols=["A", "B", "E", "F", "G", "H", "I", "R", "X"],
    usecols=[0, 1, 3, 4, 5, 6, 7, 8, 17, 23, 24],
)

In [11]:
# Only 3 people have are 1 year older than the displayed age (vs the calc age).
# Age is fine to use.
patient_df[patient_df["CalcAge"] - patient_df["Age"] >= 1]

,ID,REDCapID,StudyDate,DOB,Age,Sex,Height,Weight,PartitionKey,CalcAgeExact,CalcAge
101,28-40,28-40,2022-10-24,1979-10-08,42.0,Female,166.0,105.0,f6648353-330f-45f5-9390-0c4b7da9bef3,43.043806,43
257,173,173,2019-12-13,1994-11-17,24.0,Female,167.0,67.0,b5459a25-ce81-4205-a319-e8bbcb514fdb,25.071184,25
338,293,293,2020-09-29,1991-08-26,28.0,Female,166.0,70.2,7d47a724-c741-4d2f-a402-cbcf7d425cd8,29.091547,29


## Analyse ID vs RedCap ID

In [22]:
patient_df = pd.read_csv(
    dh.get_path_to_main()
    + "DataFiles/BR/REDCapData/ProcessedData/brPatient_20240509.csv",
    # usecols=["A", "B", "E", "F", "G", "H", "I", "R"],
    usecols=[0, 1, 4, 5, 6, 7, 8, 17],
)

In [24]:
patient_df.dtypes

ID                int64
REDCapID         object
DOB              object
Age             float64
Sex              object
Height          float64
Weight          float64
PartitionKey     object
dtype: object

In [53]:
idmap = pd.read_excel(
    dh.get_path_to_main()
    + "DataFiles/BR/REDCapData/IDMappingFiles/PatientIDMappingFile-20240509.xlsx",
    dtype={"ID": str, "redcap_id": str},
)
# idmap['ID'] = idmap['ID'].astype(str)
# idmap['redcap_id'] = idmap['redcap_id'].astype(str)
idmap.dtypes
# patient_df.merge(idmap, left_on='ID', right_on='ID')

ID           object
redcap_id    object
dtype: object

In [54]:
raw_df = pd.read_csv(
    dh.get_path_to_main()
    + "DataFiles/BR/REDCapData/DataExportFiles/AnalysisOfRemoteMoni_DATA_2023-11-29_1445.csv",
    # usecols=["A", "B", "E", "F", "G", "H", "I", "R"],
    usecols=[0, 1],
    dtype={"study_id": str},
)
# raw_df['study_id'] = raw_df['study_id'].astype(str)

In [55]:
unified_df = raw_df.merge(idmap, left_on="study_id", right_on="redcap_id")
print(unified_df.shape)
print(unified_df.ID.unique())

(21578, 4)
['316' '317' '318' '319' '320' '321' '322' '323' '324' '325' '326' '327'
 '328' '329' '330' '331' '332' '333' '334' '335' '336' '337' '338' '339'
 '340' '341' '342' '343' '344' '345' '346' '347' '348' '349' '350' '359'
 '360' '361' '376' '377' '378' '379' '380' '381' '382' '383' '384' '351'
 '352' '353' '354' '355' '356' '357' '358' '362' '363' '364' '365' '366'
 '367' '368' '369' '370' '385' '386' '387' '371' '372' '373' '374' '375'
 '388' '389' '390' '391' '392' '393' '394' '395' '396' '397' '398' '399'
 '400' '401' '402' '403' '404' '405' '406' '407' '408' '409' '410' '411'
 '412' '413' '414' '415' '416' '417' '418' '419' '420' '421' '422' '423'
 '424' '425' '426' '427' '428' '429' '430' '431' '432' '433' '434' '435'
 '436' '437' '438' '439' '440' '441' '442' '443' '444' '445' '446' '447'
 '448' '449' '450' '451' '452' '453' '454' '455' '456' '101' '102' '103'
 '104' '105' '106' '107' '108' '109' '110' '111' '112' '113' '114' '115'
 '116' '117' '118' '119' '120' '121' '12

## Drug therapy

In [2]:
drug_df = bd.load_drug_therapies()

ERROR:root:ID 175 - ?? Symkevi start date is wrong, removing it because no clue about the true date (maybe 2019?)
ERROR:root:ID 206 - Updating Ivacaftor stop date to not overlap and changing Ivacaftor + Symkevi to Trikafta


In [11]:
# Find latest DrugTherapyType for each patient
latest_drug_df = drug_df.groupby("ID").first().reset_index()
latest_drug_df.DrugTherapyType.value_counts()

DrugTherapyType
Trikafta     398
Symkevi       19
Ivacaftor     14
Unknown        4
Name: count, dtype: int64

In [10]:
# Total values counts
drug_df["DrugTherapyType"].value_counts()

DrugTherapyType
Trikafta     427
Symkevi      137
Ivacaftor     38
Orkambi        7
Unknown        5
Name: count, dtype: int64

In [4]:
# Test adding drug therapy info to a dataframe
df = pd.read_excel(
    dh.get_path_to_main() + "/ExcelFiles/BR/BR_O2_FEV1_FEF2575_PEF_Nan.xlsx",
    dtype={"ID": str},
)
df["Date Recorded"] = pd.to_datetime(df["Date Recorded"]).dt.date

In [5]:
df1 = bd.add_drug_therapy_to_df(df)

In [6]:
df1.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,PEF (L/s),ecFEV1,ecFEF2575,ecPEF (L/s),Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,DrugTherapyType
0,101,2019-01-25,1.31,97.0,0.54,NaN,NaN,1.31,0.67,NaN,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,None
1,101,2019-01-26,1.31,98.0,0.57,NaN,NaN,1.31,0.67,NaN,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,None
2,101,2019-01-27,1.31,96.0,0.67,NaN,NaN,1.31,0.69,NaN,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,None
3,101,2019-01-28,1.30,96.0,0.69,NaN,NaN,1.31,0.69,NaN,Male,173.0,53,3.610061,97.150104,36.287474,36.010470,98.816157,None
4,101,2019-01-29,1.28,98.0,0.60,NaN,NaN,1.30,0.69,NaN,Male,173.0,53,3.610061,97.150104,36.010470,35.456463,100.874827,None
